In [1]:
import pandas as pd 
from scholarly import scholarly

In [2]:
data = pd.read_excel('data/fnfr_transformation_chercheurs-scholar.xlsx')
data = data[[
    'Titre de la demande / Application Title', 'Concours / Award',
    'Chercheur-e principal-e désigné-e / Nominated Principal Investigator (PI)',
    'profil scholar',
]]

data = data.to_dict('records')
pd.DataFrame(data)

,Titre de la demande / Application Title,Concours / Award,Chercheur-e principal-e désigné-e / Nominated Principal Investigator (PI),profil scholar
0,Rare Isotopes to Transform Cancer Therapy,2022,"Bénard, François",https://scholar.google.ca/citations?user=iZ2An...
1,Enabling novel cardiac therapies with pluripot...,2022,"Laflamme, Michael",https://scholar.google.ca/citations?user=ZmBHY...
2,Abundant Intelligences: Expanding Artificial I...,2022,"Lewis, Jason",https://scholar.google.com/citations?user=Yd-p...
3,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Menon, Ravi",https://scholar.google.com/citations?user=1Jrr...
4,A pipeline approach for the rational design of...,2022,"Schuurmans, Carol",https://scholar.google.ca/citations?user=hqwq5...
5,CANSTOREnergy: Seasonal storage of renewable e...,2022,"Sinton, David",https://scholar.google.ca/citations?user=Dgdo6...
6,BIOSCAN: Tracing the Patterns of Life on a Cha...,2020,"Hebert, Paul",https://scholar.google.com/citations?user=VScb...
7,Ărramăt: Strengthening Health and Well-Being t...,2020,"Parlee, Brenda",https://scholar.google.ca/citations?user=QRcU4...
8,Mend the Gap: A Transformative Biomaterials Pl...,2020,"Madden, John",https://scholar.google.ca/citations?user=SOkJC...
9,Inclusive Design for Employment Access (IDEA),2020,"Tompa, Emile",https://scholar.google.com/citations?user=nuGB...


In [9]:
for x in data:
    id = x['profil scholar'].split('=')[1][:-3]
    res = scholarly.search_author_id(id)
    affiliation = res['affiliation']
    interests = res['interests']

    x['affiliation'] = affiliation
    x['interests'] = interests

pd.DataFrame(data).to_csv('data/fnfr_scholarly__api_PIs.csv', index=False)
data_pis = pd.DataFrame(data).rename(
    columns={
        'Titre de la demande / Application Title': 'projet',
        'Concours / Award': 'concours',
        'Chercheur-e principal-e désigné-e / Nominated Principal Investigator (PI)': 'chercheur',
    }
)

___

In [22]:
data_cochercheurs = pd.read_excel('data/fnfr_scholarly_api.xlsx').drop(columns='coauthors')
data_cochercheurs = data_cochercheurs.rename(
    columns= {
        'établissement': 'affiliation'
    }
    )

data = pd.concat([data_pis, data_cochercheurs])
data = data.sort_values(by=['chercheur', 'profil scholar'])
data[data['chercheur'] == 'Lewis, Jason']
data.to_csv('data/fnfr_scholarly_api.csv')

_________________________________________

In [6]:
data = pd.read_csv('data/affiliationsChercheurs.csv', sep=';')
data = data[['projet', 'concours', 'chercheur', 'établissement']]

data = data.to_dict('records')
pd.DataFrame(data)

,projet,concours,chercheur,établissement
0,A pipeline approach for the rational design of...,2022,"Götz, Magdalena","Biomedical Center, Ludwig Maximilian Universit..."
1,A pipeline approach for the rational design of...,2022,"Stricker, Stefan",LMU Munich
2,A pipeline approach for the rational design of...,2022,"Choque, Maria Eugenia",Aymara Nation (Region 43)
3,A pipeline approach for the rational design of...,2022,"Badea, Ildiko",University of Saskatchewan
4,A pipeline approach for the rational design of...,2022,"Bethell, Jennifer",University Health Network
...,...,...,...,...
620,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Sullivan, Jacqueline",Western University
621,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Watts, Joel",University of Toronto
622,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Williams, Sylvain",Université McGill
623,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Lipton, Mark",Cambridge Brain Sciences


In [10]:
# Retrieve the author's data, fill-in, and print
for x in data:
    author = x['chercheur'].split(',')
    author = " ".join([author[-1], author[0]]).strip()
    etablissement = x['établissement']
    query = f'{author} {etablissement}'

    search_query = scholarly.search_author(query)
    
    try:
        author_info = next(search_query)
        
    except StopIteration:
        search_query = scholarly.search_author(author)

        try:
            author_info = next(search_query)

        except StopIteration:
            print(f"No author found for: {x['chercheur']}")
            continue
    
    author = scholarly.fill(author_info)
    
    x['affiliation'] = author['affiliation']
    x['interests'] = author['interests']
    x['coauthors'] = author['coauthors']

pd.DataFrame(data)

No author found for: Bethell, Jennifer
No author found for: Gibbings, Derrick
No author found for: Jacobs-LeVan, Julia
No author found for: Morshead, Cindi
No author found for: Running Wolf, Caroline
No author found for: Benesiinaabandan, Scott
No author found for: Little Bear, Leroy
No author found for: Nagam, Julie
No author found for: Pechawis, Archer
No author found for: Ubalijoro, Eliane
No author found for: Leween, Jackson
No author found for: Enos, Kamuela
No author found for: Harrell, D. Fox
No author found for: meyer, manulani
No author found for: Amos, Kelsey
No author found for: First Rider, Amethyst
No author found for: Grimm, Holly
No author found for: Kealoha, Donavan
No author found for: Witehira, Johnson
No author found for: Whaanga, Hēmi
No author found for: Temara, Pou
No author found for: Crowshoe, Reg
No author found for: Lepine, Melody
No author found for: Littlechild, Josh
No author found for: Provost, Ira
No author found for: Yellowbird, Kacey
No author found for

,projet,concours,chercheur,établissement,affiliation,interests,coauthors
0,A pipeline approach for the rational design of...,2022,"Götz, Magdalena","Biomedical Center, Ludwig Maximilian Universit...","Professor, Department of Physiology, Developme...",[Developmental Biology],[]
1,A pipeline approach for the rational design of...,2022,"Stricker, Stefan",LMU Munich,"LMU München, Helmholtz Zentrum","[CRISPR, epigenetics, epigenome editing, neura...","[{'container_type': 'Author', 'filled': [], 's..."
2,A pipeline approach for the rational design of...,2022,"Choque, Maria Eugenia",Aymara Nation (Region 43),Docente,[ciencias de la obstetricia y neonatologia],[]
3,A pipeline approach for the rational design of...,2022,"Badea, Ildiko",University of Saskatchewan,University of Saskatchewan,"[nanotechnology, drug delivery, biomaterials]",[]
4,A pipeline approach for the rational design of...,2022,"Bethell, Jennifer",University Health Network,NaN,NaN,NaN
...,...,...,...,...,...,...,...
620,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Sullivan, Jacqueline",Western University,"Associate Professor of Philosophy, University ...","[Philosophy of Neuroscience, Philosophy of Sci...","[{'container_type': 'Author', 'filled': [], 's..."
621,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Watts, Joel",University of Toronto,"Associate Professor, University of Toronto","[Neurodegenerative diseases, protein misfoldin...",[]
622,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Williams, Sylvain",Université McGill,"McGill University, professor of psychiatry",[neuroscience and memory],"[{'container_type': 'Author', 'filled': [], 's..."
623,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Lipton, Mark",Cambridge Brain Sciences,Purdue University,[],[]


In [78]:
data = pd.DataFrame(data)
data = data[~((data['interests'].astype(str) == '[]') & (data['coauthors'].astype(str) == '[]'))]
data = data.dropna(subset=['interests', 'coauthors'])

## Avant de l'exporter: merger avec les infos obtenus avec l'id directement (pour les PIs) ; retirer les doublons
# (ie trier par url scholar et garder celui ayant une url consignée)

data.to_csv('data/fnfr_scholarly_api.csv', index=False)

In [79]:
data

,projet,concours,chercheur,affiliation,interests,coauthors
0,A pipeline approach for the rational design of...,2022,"Götz, Magdalena","Professor, Department of Physiology, Developme...",[Developmental Biology],[]
1,A pipeline approach for the rational design of...,2022,"Stricker, Stefan",university of new mexico,[biology],[]
2,A pipeline approach for the rational design of...,2022,"Choque, Maria Eugenia",Docente,[ciencias de la obstetricia y neonatologia],[]
3,A pipeline approach for the rational design of...,2022,"Badea, Ildiko",University of Saskatchewan,"[nanotechnology, drug delivery, biomaterials]",[]
5,A pipeline approach for the rational design of...,2022,"Blakney, Anna","Assistant Professor, University of British Col...","[Gene delivery, RNA vaccines, Biomaterials, Fo...",[]
...,...,...,...,...,...,...
619,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Schmitz, Taylor","Assistant Professor, Department of Physiology ...","[Attention, Memory, Aging, Alzheimer's, Multim...","[{'container_type': 'Author', 'filled': [], 's..."
620,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Sullivan, Jacqueline","Associate Professor of Philosophy, University ...","[Philosophy of Neuroscience, Philosophy of Sci...","[{'container_type': 'Author', 'filled': [], 's..."
621,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Watts, Joel","Associate Professor, University of Toronto","[Neurodegenerative diseases, protein misfoldin...",[]
622,TRIDENT: TRanslational Initiative to DE-risk N...,2022,"Williams, Sylvain","McGill University, professor of psychiatry",[neuroscience and memory],"[{'container_type': 'Author', 'filled': [], 's..."
